In [1]:
# 0. generate random order of edges
# 1. so that it actually starts with building a tree
# 2. build EPPDC for this tree
# now we iteratively add an edge (a,b)
# 4. bruteforce one direction as in the usual PPDC
# 5. bruteforce all sequences Q = b v1 v2 ... vt \inf
# 6. select the sequence which would give an EPPDC
# ...
# 7. profit! (TODO: prove that process never fails)

from collections import defaultdict
from copy import deepcopy
from enum import Enum
import random
import time

from unionfind import unionfind

In [2]:
# stuff

def gen_edges(n):
    edges = []
    for i in range(n):
        for j in range(i + 1, n):
            edges.append((i, j))
    random.shuffle(edges)
    return edges

def sort_edges_to_build_a_tree(edges, n, verbose=False):
    u = unionfind(n)
    tree_edges = []
    non_tree_edges = []
    for edge in edges:
        v1, v2 = edge
        if u.issame(v1, v2):
            non_tree_edges.append((v1, v2))
        else:
            u.unite(v1, v2)
            tree_edges.append(edge)
    random.shuffle(non_tree_edges)
    if verbose:
        print(f'{tree_edges=}')
    assert (len(tree_edges) == n - 1)
    return tree_edges, non_tree_edges


class Graph():
    def __init__(self, n, edges):
        self.n = n
        self.edges = edges
        self.neibs = dict()
        for v in range(n):
            self.neibs[v] = []
        for v1, v2 in edges:
            self.neibs[v1].append(v2)
            self.neibs[v2].append(v1)


class EdgePath():
    def __init__(self):
        self.edges = []
        self._vertices_set = set()
        self._start = None
        self._finish = None
        self.ends = set([self._start, self._finish])

    def _reverse(self):
        self.edges.reverse()
        for edge_idx, edge in enumerate(self.edges):
            self.edges[edge_idx] = (edge[1], edge[0])
        self._start, self._finish = self._finish, self._start

    def _update_start(self, vertex):
        self._start = vertex
        self.ends = set([self._start, self._finish])

    def _update_finish(self, vertex):
        self._finish = vertex
        self.ends = set([self._start, self._finish])

    def append(self, edge):
        if self.edges:
            assert edge[1] not in self._vertices_set
            if self._start == edge[0]:
                self._reverse()
            assert self._finish == edge[0]
        self.edges.append(edge)
        for vertex in edge:
            self._vertices_set.add(vertex)
        if self._start is None:
            self._start = self.edges[0][0]
        self._update_finish(edge[-1])

    def shrink(self):
        assert self.edges is not None
        assert len(self.edges) > 0
        self._vertices_set.remove(self._finish)
        self.edges.pop()
        assert len(self.edges) > 0
        self._update_finish(self.edges[-1][-1])

    def rebuild(self, new_vertices):
        self.edges = []
        self._vertices_set = set()
        self._start = None
        self._finish = None
        self.ends = set([self._start, self._finish])
        for i in range(len(new_vertices) - 1):
            self.append((new_vertices[i], new_vertices[i + 1]))

    def __contains__(self, vertex):
        return vertex in self._vertices_set

    def get_vertices(self):
        vertices = []
        for edge in self.edges:
            vertices.append(edge[0])
        vertices.append(self._finish)
        return vertices

    def __repr__(self):
        return str(self.get_vertices())


class EPPDC():
    def __init__(self, n):
        self.n = n
        self.paths = []

    def append(self, path: EdgePath):
        self.paths.append(path)

    def is_connected(self):
        u = unionfind(n)
        for path in self.paths:
            u.unite(*path.ends)
        return len(u.groups()) == 1

    def is_eppdc(self):
        if not self.is_connected():
            return False
        end_counts = defaultdict(int)
        edge_counts = defaultdict(int)
        for path in self.paths:
            for end in path.ends:
                end_counts[end] += 1
            for edge in path.edges:
                sorted_edge = edge
                if sorted_edge[0] > sorted_edge[1]:
                    sorted_edge = (edge[1], edge[0])
                edge_counts[sorted_edge] += 1
        for end, count in end_counts.items():
            if count != 2:
                print(f'fail in END counts; {end=}, {count=}')
                for path in self.paths:
                    print(f'{path=}')
                return False
        for edge, count in edge_counts.items():
            if count != 2:
                print(f'fail in EDGE counts; {edge=}, {count=}')
                return False
        return True

    def __iter__(self):
        return self.paths.__iter__()

    def __next__(self):
        return self.paths.__next__()

    def _comp(self, other):
        verts1 = []
        for path in self.paths:
            verts1.append(path.get_vertices())
        verts1.sort()
        for verts in verts1:
            if list(reversed(verts)) < verts:
                verts.reverse()
        
        verts2 = []
        for path in other.paths:
            verts2.append(path.get_vertices())
        verts2.sort()
        for verts in verts2:
            if list(reversed(verts)) < verts:
                verts.reverse()

        for idx in range(len(verts1)):
            if verts1[idx] < verts2[idx]:
                return -1
            elif verts1[idx] > verts2[idx]:
                return 1
        return 0

    def __eq__(self, other):
        return self._comp(other) == 0

    def __lt__(self, other):
        return self._comp(other) == -1

    def __hash__(self):
        verts1 = []
        for path in self.paths:
            verts1.append(path.get_vertices())
        verts1.sort()
        for idx, verts in enumerate(verts1):
            if list(reversed(verts)) < verts:
                verts.reverse()
            verts1[idx] = tuple(verts)
        return hash(tuple(verts1))


class DFSState():
    def __init__(self, n):
        self.visited = set()
        self.eppdc = EPPDC(n)
        self.cur_path = EdgePath()

    def dfs(self, v1, graph):
        self.visited.add(v1)

        # traversing children
        for v2 in graph.neibs[v1]:
            if v2 in self.visited:
                continue
            self.cur_path.append((v1, v2))
            self.dfs(v2, graph)
            self.cur_path.append((v2, v1))

        # leaving vertex v1
        self.eppdc.append(self.cur_path)
        self.cur_path = EdgePath()

    def knuth(self, v1, graph, mode='bfs'):
        assert mode in ['bfs', 'dfs']
        next_mode = 'dfs'
        if mode == 'dfs':
            next_mode = 'bfs'

        self.visited.add(v1)

        if mode == 'bfs':
            if v1 != 0:
                self.eppdc.append(self.cur_path)
                self.cur_path = EdgePath()

        # traversing children
        for v2 in graph.neibs[v1]:
            if v2 in self.visited:
                continue
            self.cur_path.append((v1, v2))
            self.knuth(v2, graph, next_mode)
            self.cur_path.append((v2, v1))

        # leaving vertex v1
        if mode == 'dfs' or v1 == 0:
            self.eppdc.append(self.cur_path)
            self.cur_path = EdgePath()


def build_tree_eppdc(graph, n, method='dfs'):
    # idea: do some kind of dsf-like tree traversal: bfs, dfs, or Knuth
    # for simplicity we go with dfs
    # also: here we introduce the data structures, useful for describing EPPDC

    # algorithm: start at root, root will be 1 end of path
    # go 1 step deeper, add edge to path
    # go further, until the leaf
    # leaf will be a second end
    # start a new path at the leaf
    # continue in this manner
    # actual algorithm is: we start a new path each time we need to leave the vertex
    state = DFSState(n)
    assert method in ['dfs', 'knuth']
    if method == 'dfs':
        state.dfs(0, graph)
    else:
        state.knuth(0, graph, 'bfs')
    return state.eppdc

class State(Enum):
    SUCCESS = 0
    FAIL = 1
    BACKTRACK = 2

In [14]:
# THE ALGORITHM ITSELF
def extend_with_backtrack(n, v1, v2, eppdc, part,
                          use_backtrack=False, verbose=False,
                          rnd_gen=None,
                          count_switches=True,
                          check_connectivity=True):
    # we want to add edge (v1, v2),
    # extend some path ending at v1 to v2

    original_v1 = v1
    checked_switches = defaultdict(int)
    switch_path = []

    while True:
        switch_path.append(v1)
        # search the paths, ending with v1
        v1_paths = []
        for possible_v1_path in eppdc:
            if v1 in possible_v1_path.ends:
                v1_paths.append(possible_v1_path)

        if rnd_gen is None:
            random.shuffle(v1_paths)  # FIXME: remove randomness; even though it is important here
        else:
            rnd_gen.shuffle(v1_paths)

        switched = False
        for path in v1_paths:
            if v2 not in path:
                path.append((v1, v2))
                if eppdc.is_connected():
                    if part == 2:
                        if not eppdc.is_eppdc():
                            print('BUG')
                    return (State.SUCCESS, (v1, v2), switch_path)
                else:
                    path.shrink()
                    if verbose:
                        print(f'  part{part}.x: {v2} not in path {path}, but ppdc not connected')
            else:
                # toy example
                # v1, v2 = (4, 3)
                # path = [2, 3, 1, 4]
                # v2 is in path
                # => new path:
                # [2, 3, 4, 1]
                # new v1, v2 = (1, 3)

                path_vertices = path.get_vertices()
                if path_vertices[0] == v1:
                    path_vertices.reverse()
                if path_vertices[-2] == v2:
                    # try to backtrack

                    # last condition is just a technicality/optimization
                    if use_backtrack and (v1 != original_v1) and len(path_vertices) >= 3:
                        if verbose:
                            print(f'  part{part}.x: try to backtrack')
                        new_path_vertices = path_vertices[:-1]
                        if verbose:
                            print(f'  before backtrack rebuild')
                            for debug_path in eppdc.paths:
                                print(f'    {debug_path=}')
                        path.rebuild(new_path_vertices)
                        if not eppdc.is_connected():
                            if verbose:
                                print('backtrack failed')
                            path.rebuild(path_vertices)
                            continue
                        else:
                            switched = True
                            if verbose:
                                print(path_vertices, '=>', new_path_vertices)
                                print(f'  BACKTRACKED_SUCCESS; after backtrack rebuild')
                                for debug_path in eppdc.paths:
                                    print(f'    {debug_path=}')
                            return (State.BACKTRACK, (v1, v2), switch_path)
                    else:
                        if v1 != original_v1:
                            if verbose:
                                print(f'  part{part}.x: could try to backtrack edge ({v1}, {v2}) for {path_vertices}')
                        continue

                new_path_vertices = []
                for v in path_vertices:
                    new_path_vertices.append(v)
                    if v == v2:
                        break
                for v in reversed(path_vertices):
                    if v == v2:
                        break
                    new_path_vertices.append(v)

                if verbose:
                    print('before rebuild')

                new_v1 = new_path_vertices[-1]

                if count_switches:
                    if checked_switches[(v1, new_v1)] >= 2:
                        continue
                    if checked_switches[(new_v1, v1)] >= 2:
                        continue
                else:
                    if checked_switches[(v1, new_v1)] >= 10:
                        continue
                    if checked_switches[(new_v1, v1)] >= 10:
                        continue
                checked_switches[(v1, new_v1)] += 1

                if check_connectivity:
                    assert eppdc.is_connected()
                path.rebuild(new_path_vertices)
                if check_connectivity:
                    if not eppdc.is_connected():
                        if verbose:
                            print(f'tried {path_vertices} => {new_path_vertices}, but ppdc not connected')
                        path.rebuild(path_vertices)
                        assert eppdc.is_connected()
                        continue

                if verbose:
                    print('after rebuild')
                    print(path_vertices, '=>', new_path_vertices)
                    print(f'  current paths')
                    for debug_path in eppdc.paths:
                        print(f'{debug_path=}')

                switched = True
                v1 = new_v1
                break

        if switched:
            if verbose:
                print(f'  part{part}.x: adding edge ({v1}, {v2})')
        else:
            break

    return (State.FAIL, (v1, v2), None)

In [16]:
# start from graph

def parse_paths(n, paths_str):
    import ast
    eppdc = EPPDC(n)
    paths_str = paths_str.strip().split('\n')
    for path in paths_str:
        path_vertices = ast.literal_eval(path.strip().replace('path=', ''))
        path = EdgePath()
        path.rebuild(path_vertices)
        eppdc.append(path)
    assert eppdc.is_eppdc()
    return eppdc

verbose = True

print_stats = True
# print_stats = False

check_edge_idx = -1

backtrack_iteration = 20

max_rep = 1

prev_max_backtrack_count = 0
max_backtrack_count = 0
max_backtracked_edges = []
max_backtracked_edges_all = []

for rep0 in range(max_rep):
    print(rep0, '/', max_rep)

    n=11; seed=848962982; rep0=91; method='knuth'; rep1=1
    
    random.seed(seed)
    edges = gen_edges(n)
    tree_edges, non_tree_edges = sort_edges_to_build_a_tree(edges, n, verbose=verbose)
    random.seed(seed + rep0)
    for edge_idx, (orig_v1, orig_v2) in enumerate(non_tree_edges):
        if random.random() > 0.5:
            non_tree_edges[edge_idx] = (orig_v2, orig_v1)

    tmp_random_generator = random.Random(seed + rep0 + rep1)

    selected_edge_idx = 12
    adding_edge = (8, 10)
    paths_str = '''
        path=[0, 2, 6, 10, 7]
        path=[6, 8, 5]
        path=[4, 8, 9, 3, 5]
        path=[4, 7, 3, 5, 8, 2]
        path=[9, 8, 4, 7, 2]
        path=[7, 8, 6, 3, 2, 1]
        path=[8, 1, 2, 6, 3, 7, 10, 0]
        path=[1, 9, 3]
        path=[9, 1, 8, 7, 2, 10]
        path=[6, 10, 2, 3]
        path=[8, 2, 0, 10]
    '''
    eppdc = parse_paths(n, paths_str)

    len_non_tree_edges = len(non_tree_edges)
    for edge_idx in range(len_non_tree_edges):
        if edge_idx < selected_edge_idx:
            continue
        if edge_idx > selected_edge_idx:
            break

        eppdc_before_part1 = deepcopy(eppdc)

        v1, v2 = adding_edge

        iteration_count = 0
        backtracked_edges = [(v1, v2)]
        backtracked_edges_all = []
        backtracked_once = False
        use_backtrack1 = False
        use_backtrack2 = False
        common_vertex = None

        while True:  # FIXME: we need an explicit mechanism to not have an infinite loop
            success1 = State.FAIL
            success2 = State.FAIL

            iteration_count += 1
            if (iteration_count > backtrack_iteration):
                if iteration_count % 4 < 2:
                    use_backtrack1 = False
                    use_backtrack2 = True
                    if verbose:
                        print('TURN ON BACKTRACK2')
                else:
                    use_backtrack1 = True
                    use_backtrack2 = False
                    if verbose:
                        print('TURN ON BACKTRACK1')
#             if (iteration_count > backtrack_iteration * 5):
#                 assert False

            eppdc = deepcopy(eppdc_before_part1)

            if verbose:
                print(f'{edge_idx=} before part 1; adding edge {(v1, v2)}')
                for path in eppdc:
                    print(path)

            eppdc = deepcopy(eppdc_before_part1)
            if not use_backtrack1:
                success1, (new_v1_p1, new_v2_p1), _ = extend_with_backtrack(n, v1, v2, eppdc, 1,
                                                                            use_backtrack=use_backtrack1,
                                                                            verbose=verbose)
            else:
                success1, (new_v1_p1, new_v2_p1), _ = extend_with_backtrack(n, v1, v2, eppdc, 1,
                                                                            use_backtrack=use_backtrack1,
                                                                            verbose=verbose,
                                                                            check_connectivity=False)
            if verbose:
                print(f'{edge_idx=} part1 success: {success1}')
                if success1 != State.FAIL:
                    print(f'{success1=}')
                    print(f'{edge_idx=} after part 1;')
                    for path in eppdc:
                        print(path)
                    print(f'adding edge {(v2, v1)}')

            if success1 == State.FAIL:
#                 v1, v2 = v2, v1  # looks like we need this; even when we backtracked
                continue

            success2, (new_v2_p2, new_v1_p2), _ = extend_with_backtrack(n, v2, v1, eppdc, 2,
                                                                        use_backtrack=use_backtrack2,
                                                                        verbose=verbose)
                                                                       # check_connectivity=False  # TODO

            if success2 == State.FAIL:
                success1 = State.FAIL

            if verbose:
                print(f'{edge_idx=} part2 success: {success2}')    
                if success2 == State.SUCCESS:
                    print(f'{edge_idx=} after part 2')
                    for path in eppdc:
                        print(path)
                time.sleep(0.1)

            if success2 != State.FAIL:
                assert eppdc.is_connected()
                assert eppdc.is_eppdc()

                if success1 == State.SUCCESS and success2 == State.SUCCESS:
                    # assert not use_backtrack1 and not use_backtrack2  # FIXME: ideally this should be the case
                    backtrack_count = len(list(backtracked_edges)) - 1
                    if backtrack_count >= max_backtrack_count:
                        max_backtrack_count = backtrack_count
                        max_backtracked_edges = backtracked_edges
                        max_backtracked_edges_all = backtracked_edges_all
#                         print(f'{edge_idx=}')
#                         print(f'{max_backtracked_edges=}')
#                         print(f'{max_backtracked_edges_all=}')
#                         print(f'{max_backtrack_count=}')
                    break

                assert (success1 == State.BACKTRACK or success2 == State.BACKTRACK)
                if success1 == State.BACKTRACK:
                    new_v1, new_v2 = new_v1_p1, new_v2_p1
                    if verbose:
                        print(f'{rep0=} {edge_idx=} PART1 backtrack!')
                else:
                    new_v1, new_v2 = new_v2_p2, new_v1_p2
                    if verbose:
                        print(f'{rep0=} {edge_idx=} part2 backtrack!')
                new_edge = (new_v1, new_v2)
                if common_vertex is None:
                    common_vertex = set(backtracked_edges[0]) & set(new_edge)
                    assert len(common_vertex) == 1
                    common_vertex = list(common_vertex)[0]

                if new_v1 != common_vertex and new_v2 != common_vertex:
                    continue

#                 if (new_v1, new_v2) in backtracked_edges and backtracked_edges[-2] == (new_v1, new_v2):
#                     backtracked_edges = backtracked_edges[:-1]
#                 else:
                backtracked_edges.append((new_v1, new_v2))
                backtracked_edges_all.append(((v1, v2), (new_v1, new_v2)))

                if verbose:
                    print(f'{edge_idx=} backtrack edge ({v1}, {v2}); now try ({new_v1}, {new_v2})')
                    print('')

                v1, v2 = new_v1, new_v2
                eppdc_before_part1 = deepcopy(eppdc)
                iteration_count = 0
                use_backtrack1 = False
                use_backtrack2 = False
                backtracked_once = True
            else:
                if verbose:
                    print('fail, try again')
                    print('')

        if success2 == State.FAIL:
            print(f'{n=}; {seed=}; {rep0=}; {method=}')
            print(f'{success1=}, {success2=}, {edge_idx=} vs {len(non_tree_edges)}')
            print(f'(v1, v2) = ({v1}, {v2})')
            for path in eppdc_before_part1:
                print(path)
            assert False
        assert eppdc.is_eppdc()

    if max_backtrack_count > prev_max_backtrack_count:
        print(f'{n=}; {seed=}; {rep0=}; {method=}')
        print(f'{max_backtracked_edges=}')
        print(f'{max_backtracked_edges_all=}')
        print(f'{max_backtrack_count=}')
    prev_max_backtrack_count = max_backtrack_count

print('DONE')

0 / 1
tree_edges=[(6, 8), (3, 6), (2, 6), (4, 7), (5, 8), (1, 2), (4, 8), (1, 9), (2, 10), (0, 2)]
edge_idx=12 before part 1; adding edge (8, 10)
[0, 2, 6, 10, 7]
[6, 8, 5]
[4, 8, 9, 3, 5]
[4, 7, 3, 5, 8, 2]
[9, 8, 4, 7, 2]
[7, 8, 6, 3, 2, 1]
[8, 1, 2, 6, 3, 7, 10, 0]
[1, 9, 3]
[9, 1, 8, 7, 2, 10]
[6, 10, 2, 3]
[8, 2, 0, 10]
before rebuild
after rebuild
[10, 0, 2, 8] => [10, 8, 2, 0]
  current paths
debug_path=[0, 2, 6, 10, 7]
debug_path=[6, 8, 5]
debug_path=[4, 8, 9, 3, 5]
debug_path=[4, 7, 3, 5, 8, 2]
debug_path=[9, 8, 4, 7, 2]
debug_path=[7, 8, 6, 3, 2, 1]
debug_path=[8, 1, 2, 6, 3, 7, 10, 0]
debug_path=[1, 9, 3]
debug_path=[9, 1, 8, 7, 2, 10]
debug_path=[6, 10, 2, 3]
debug_path=[10, 8, 2, 0]
  part1.x: adding edge (0, 10)
before rebuild
after rebuild
[10, 8, 2, 0] => [10, 0, 2, 8]
  current paths
debug_path=[0, 2, 6, 10, 7]
debug_path=[6, 8, 5]
debug_path=[4, 8, 9, 3, 5]
debug_path=[4, 7, 3, 5, 8, 2]
debug_path=[9, 8, 4, 7, 2]
debug_path=[7, 8, 6, 3, 2, 1]
debug_path=[8, 1, 2, 6, 3

fail, try again

edge_idx=12 before part 1; adding edge (7, 10)
[0, 2, 6, 10, 8]
[6, 8, 5]
[4, 8, 9, 3, 5]
[4, 7, 3, 5, 8, 2]
[9, 8, 4, 7, 2]
[1, 2, 3, 6, 8, 7]
[0, 10, 8, 1, 2, 6, 3, 7]
[1, 9, 3]
[9, 1, 8, 7, 2, 10]
[6, 10, 2, 3]
[8, 2, 0, 10]
before rebuild
after rebuild
[0, 10, 8, 1, 2, 6, 3, 7] => [0, 10, 7, 3, 6, 2, 1, 8]
  current paths
debug_path=[0, 2, 6, 10, 8]
debug_path=[6, 8, 5]
debug_path=[4, 8, 9, 3, 5]
debug_path=[4, 7, 3, 5, 8, 2]
debug_path=[9, 8, 4, 7, 2]
debug_path=[1, 2, 3, 6, 8, 7]
debug_path=[0, 10, 7, 3, 6, 2, 1, 8]
debug_path=[1, 9, 3]
debug_path=[9, 1, 8, 7, 2, 10]
debug_path=[6, 10, 2, 3]
debug_path=[8, 2, 0, 10]
  part1.x: adding edge (8, 10)
before rebuild
after rebuild
[0, 10, 7, 3, 6, 2, 1, 8] => [0, 10, 8, 1, 2, 6, 3, 7]
  current paths
debug_path=[0, 2, 6, 10, 8]
debug_path=[6, 8, 5]
debug_path=[4, 8, 9, 3, 5]
debug_path=[4, 7, 3, 5, 8, 2]
debug_path=[9, 8, 4, 7, 2]
debug_path=[1, 2, 3, 6, 8, 7]
debug_path=[0, 10, 8, 1, 2, 6, 3, 7]
debug_path=[1, 9, 3]
d

In [25]:
# DONE: choose worst edge

# n = 20
# n = 13
# n = 12
# n = 11
# n = 10
n = 9
# n = 8
# n = 7
# n = 6

# method = 'dfs'
method = 'knuth'

verbose = False
# verbose = True

print_stats = True
# print_stats = False

check_edge_idx = -1

# select_worst_edge = False
select_worst_edge = True

backtrack_iteration = 20

max_rep = 10000
max_rep = 3000
max_rep = 1000
# max_rep = 100
# max_rep = 10
# max_rep = 1

rep1 = 1

prev_max_backtrack_count = 0
max_backtrack_count = 0
max_backtracked_edges = []
max_backtracked_edges_all = []

for rep0 in range(max_rep):
#     if (rep0 % 100 == 0):
#     if (rep0 % 40 == 0):
#     if (rep0 % 10 == 0):
    if (rep0 % 4 == 0):
#     if (rep0 % 1 == 0):
        print(rep0, '/', max_rep)
#     n = random.randint(5, 25)
    seed = random.randint(0, 1000000000)

#     n=11; seed=168723644; rep0=75; rep1=1

#     n=11; seed=784757882; rep0=35; rep1=1  # breaks no-backtracks-in-part1

#     n=11; seed=848962982; rep0=91; rep1=1; method = 'knuth'  # breaks v1,v2=v2,v1

#     n=9; seed=155016445; rep0=868; rep1+=1  # removed v1,v2=v2,v1; need to add "check_connectivity=False"

#     n=11; seed=572040645; rep0=120; method='knuth'; rep1=1
#     check_edge_idx = 8

#     n=6; seed=453421617; rep0=127; method='knuth'; rep1=1
    
    random.seed(seed)

    # 0. generate random order of edges
    edges = gen_edges(n)
    # 1. so that we actually start with building a tree
    tree_edges, non_tree_edges = sort_edges_to_build_a_tree(edges, n, verbose=verbose)
    random.seed(seed + rep0)

    # FIXME: this will probably break all examples above
    for edge_idx, (orig_v1, orig_v2) in enumerate(non_tree_edges):
        if random.random() > 0.5:
            non_tree_edges[edge_idx] = (orig_v2, orig_v1)

    tmp_random_generator = random.Random(seed + rep0 + rep1)

    # 2. build EPPDC for this tree
    tree = Graph(n=n, edges=tree_edges)
    tree_eppdc = build_tree_eppdc(tree, n, method=method)
    
    assert tree_eppdc.is_eppdc()
    if verbose:
        for path in tree_eppdc:
            print(path)

    # now we iteratively add an edge
    # steps 4.-6.
    eppdc = deepcopy(tree_eppdc)

#     non_tree_edges = set(non_tree_edges)
    len_non_tree_edges = len(non_tree_edges)
    for edge_idx in range(len_non_tree_edges):
#         print(f'{edge_idx=}')
#         if edge_idx == 26:
#             verbose = True

#         if edge_idx >= len_non_tree_edges * 2/3:
#         if edge_idx >= len_non_tree_edges * 1/2:
#         if edge_idx >= n:

        if edge_idx >= n * 2:
            break

        eppdc_before_part1 = deepcopy(eppdc)

        if select_worst_edge:
            selection = set()
            if edge_idx >= n/2:
#             if edge_idx >= 0:
                for path in eppdc:
                    verts = path.get_vertices()
                    if len(verts) == 2:
                        continue
#                     edge1 = (verts[0], verts[-1])
#                     edge2 = (verts[-1], verts[0])
#                     if edge1 in non_tree_edges:
#                         selection.add(edge1)
#                     elif edge2 in non_tree_edges:
#                         selection.add(edge2)

                    edge1 = (verts[0], verts[-2])
                    edge2 = (verts[-2], verts[0])
                    if edge1 in non_tree_edges:
                        selection.add(edge1)
                    elif edge2 in non_tree_edges:
                        selection.add(edge2)

                    edge1 = (verts[1], verts[-1])
                    edge2 = (verts[-1], verts[1])
                    if edge1 in non_tree_edges:
                        selection.add(edge1)
                    elif edge2 in non_tree_edges:
                        selection.add(edge2)

#                 for path in eppdc:
#                     verts = path.get_vertices()
#                     edge1 = (verts[0], verts[-1])
#                     edge2 = (verts[-1], verts[0])
#                     if edge1 in selection:
#                         selection.remove(edge1)
#                     elif edge2 in selection:
#                         selection.remove(edge2)
            selection = list(selection)

            if len(selection) == 0:
                non_tree_edges = list(non_tree_edges)
                tmp_random_generator.shuffle(non_tree_edges)
#                 selection = deepcopy(non_tree_edges[:4])
#                 selection = deepcopy(non_tree_edges[:8])
                selection = deepcopy(non_tree_edges[:12])
#                 selection = deepcopy(non_tree_edges)
    #             if edge_idx % 2 != 0:
        #         if edge_idx % 3 != 0:
    #             if edge_idx % 3 == 0:
    #                 selection = deepcopy(non_tree_edges[:1])
                non_tree_edges = set(non_tree_edges)

            # choose worst edge
            worst_edge = None
            worst_sum = None
            worst_min = None
            for tmp_v1, tmp_v2 in selection:
                part1_solutions = defaultdict(set)
                for (tv1, tv2) in [(tmp_v1, tmp_v2), (tmp_v2, tmp_v1)]:
                    part1_count_iteration_count = 0

#                     thresh0 = 40
#                     thresh0 = 30
                    thresh0 = 20
#                     thresh0 = 10
                    thresh = thresh0
                    count_switches = True

#                     if edge_idx == check_edge_idx:
#                         thresh = 300

                    while (part1_count_iteration_count < thresh):
                        part1_count_iteration_count += 1
                        eppdc = deepcopy(eppdc_before_part1)
                        tmp_success1, _, switch_path = extend_with_backtrack(n, tv1, tv2, eppdc, 1,
                                                                             use_backtrack=True,
                                                                             verbose=False,
                                                                             rnd_gen=tmp_random_generator,
                                                                             count_switches=count_switches,
#                                                                              check_connectivity=False
                                                                            )
#                         if tmp_success1 == State.SUCCESS:
                        if tmp_success1 != State.FAIL:
#                             part1_solutions[(tv1, tv2)].add(eppdc)
                            part1_solutions[(tv1, tv2)].add(tuple(switch_path))
    #                         if len(part1_solutions[(tv1, tv2)]) > 20:
#                                 break
                        if (len(part1_solutions[(tv1, tv2)]) <= 1) and \
                                (part1_count_iteration_count == thresh0):
#                         if (len(part1_solutions[(tv1, tv2)]) <= 0) and \
#                                 (part1_count_iteration_count == thresh0):
#                             thresh = thresh0 * 20
                            thresh = thresh0 * 4
                len1 = len(part1_solutions[(tmp_v1, tmp_v2)])
                len2 = len(part1_solutions[(tmp_v2, tmp_v1)])
    #             if len1 < 1 or len2 < 1:
    #                 print(f'{n=}; {seed=}; {rep0=}; {method=}; {rep1=}')
    #                 print(f'{edge_idx=}')
    #                 print('lens', len(part1_solutions[(tmp_v1, tmp_v2)]), len(part1_solutions[(tmp_v2, tmp_v1)]))
                if worst_edge is None or \
                        (worst_min > min(len1, len2)):# or \
#                         (worst_min == min(len1, len2) and (worst_sum > len1 + len2)):
#                     (worst_sum > len1 + len2):
    #         or \
                    worst_edge = (tmp_v1, tmp_v2)
                    if len1 < len2:
                        worst_edge = (tmp_v1, tmp_v2)
                    else:
                        worst_edge = (tmp_v2, tmp_v1)
                    worst_min = min(len1, len2)
                    worst_sum = len1 + len2
        else:
            worst_edge = non_tree_edges[0]

        v1, v2 = worst_edge
        if worst_edge in non_tree_edges:
            non_tree_edges.remove(worst_edge)
        else:
            non_tree_edges.remove((worst_edge[1], worst_edge[0]))

        if worst_min == 0:# and worst_sum <= 1:
#         if (worst_min <= 1) or (edge_idx == check_edge_idx):
            print(f'{n=}; {seed=}; {rep0=}; {method=}; {rep1=}')
            print(f'{edge_idx=}')
            print('worst', worst_min, worst_sum, worst_edge)
            for path in eppdc_before_part1:
                print(f'{path=}')
            if worst_sum == 0:
                print('HOLY COW!')
                
                part1_solutions = defaultdict(set)
                for (tv1, tv2) in [(v1, v2), (v2, v1)]:
                    part1_count_iteration_count = 0
                    thresh = 100
                    while (part1_count_iteration_count < thresh):
                        part1_count_iteration_count += 1
                        eppdc = deepcopy(eppdc_before_part1)
                        tmp_success1, _ = extend_with_backtrack(n, tv1, tv2, eppdc, 1,
                                                                use_backtrack=False,
                                                                verbose=False,
                                                                rnd_gen=tmp_random_generator,
                                                                count_switches=False,
                                                                check_connectivity=False)
                        if tmp_success1 == State.SUCCESS:
                            part1_solutions[(tv1, tv2)].add(eppdc)
                len1 = len(part1_solutions[(v1, v2)])
                len2 = len(part1_solutions[(v2, v1)])
                print('without connectivity', len1, len2)

        iteration_count = 0
        backtracked_edges = [(v1, v2)]
        backtracked_edges_all = []
        backtracked_once = False
        use_backtrack1 = False
        use_backtrack2 = False
        common_vertex = None

        while True:  # FIXME: we need an explicit mechanism to not have an infinite loop
            success1 = State.FAIL
            success2 = State.FAIL

            iteration_count += 1
            if (iteration_count > backtrack_iteration):
                if iteration_count % 4 < 2:
                    use_backtrack1 = False
                    use_backtrack2 = True
#                     print('HI!')
                else:
                    use_backtrack1 = True
                    use_backtrack2 = False
#                     print('hi2')
#             if (iteration_count > backtrack_iteration * 5):
#                 assert False

            eppdc = deepcopy(eppdc_before_part1)

            if verbose:
                print(f'{edge_idx=} before part 1; adding edge {(v1, v2)}')
                for path in eppdc:
                    print(path)

            eppdc = deepcopy(eppdc_before_part1)
            if not use_backtrack1:
                success1, (new_v1_p1, new_v2_p1), _ = extend_with_backtrack(n, v1, v2, eppdc, 1,
                                                                            use_backtrack=use_backtrack1,
                                                                            verbose=verbose)
            else:
                success1, (new_v1_p1, new_v2_p1), _ = extend_with_backtrack(n, v1, v2, eppdc, 1,
                                                                            use_backtrack=use_backtrack1,
                                                                            verbose=verbose,
                                                                           )
#                                                                             check_connectivity=False)
            if verbose:
                print(f'{edge_idx=} part1 success: {success1}')
                if success1 != State.FAIL:
                    print(f'{success1=}')
                    print(f'{edge_idx=} after part 1;')
                    for path in eppdc:
                        print(path)
                    print(f'adding edge {(v2, v1)}')

            if success1 == State.FAIL:
#                 v1, v2 = v2, v1  # looks like we need this; even when we backtracked
                continue

            success2, (new_v2_p2, new_v1_p2), _ = extend_with_backtrack(n, v2, v1, eppdc, 2,
                                                                        use_backtrack=use_backtrack2,
                                                                        verbose=verbose)
                                                                       # check_connectivity=False  # TODO

            if success2 == State.FAIL:
                success1 = State.FAIL

            if verbose:
                print(f'{edge_idx=} part2 success: {success2}')    
                if success2 == State.SUCCESS:
                    print(f'{edge_idx=} after part 2')
                    for path in eppdc:
                        print(path)
                time.sleep(0.1)

            if success2 != State.FAIL:
                assert eppdc.is_connected()
                assert eppdc.is_eppdc()

                if success1 == State.SUCCESS and success2 == State.SUCCESS:
                    # assert not use_backtrack1 and not use_backtrack2  # FIXME: ideally this should be the case
                    backtrack_count = len(list(backtracked_edges)) - 1
                    if backtrack_count >= max_backtrack_count:
                        max_backtrack_count = backtrack_count
                        max_backtracked_edges = backtracked_edges
                        max_backtracked_edges_all = backtracked_edges_all
#                         print(f'{edge_idx=}')
#                         print(f'{max_backtracked_edges=}')
#                         print(f'{max_backtracked_edges_all=}')
#                         print(f'{max_backtrack_count=}')
                    break

                assert (success1 == State.BACKTRACK or success2 == State.BACKTRACK)
                if success1 == State.BACKTRACK:
                    new_v1, new_v2 = new_v1_p1, new_v2_p1
                    if verbose:
                        print(f'{rep0=} {edge_idx=} PART1 backtrack!')
                else:
                    new_v1, new_v2 = new_v2_p2, new_v1_p2
                    if verbose:
                        print(f'{rep0=} {edge_idx=} part2 backtrack!')
                new_edge = (new_v1, new_v2)
                if common_vertex is None:
                    common_vertex = set(backtracked_edges[0]) & set(new_edge)
                    assert len(common_vertex) == 1
                    common_vertex = list(common_vertex)[0]

                if new_v1 != common_vertex and new_v2 != common_vertex:
                    continue

#                 if (new_v1, new_v2) in backtracked_edges and backtracked_edges[-2] == (new_v1, new_v2):
#                     backtracked_edges = backtracked_edges[:-1]
#                 else:
                backtracked_edges.append((new_v1, new_v2))
                backtracked_edges_all.append(((v1, v2), (new_v1, new_v2)))

                if verbose:
                    print(f'{edge_idx=} backtrack edge ({v1}, {v2}); now try ({new_v1}, {new_v2})')
                    print('')

                v1, v2 = new_v1, new_v2
                eppdc_before_part1 = deepcopy(eppdc)
                iteration_count = 0
                use_backtrack1 = False
                use_backtrack2 = False
                backtracked_once = True
            else:
                if verbose:
                    print('fail, try again')
                    print('')

        if success2 == State.FAIL:
            print(f'{n=}; {seed=}; {rep0=}; {method=}')
            print(f'{success1=}, {success2=}, {edge_idx=} vs {len(non_tree_edges)}')
            print(f'(v1, v2) = ({v1}, {v2})')
            for path in eppdc_before_part1:
                print(path)
            assert False
        assert eppdc.is_eppdc()

    if max_backtrack_count > prev_max_backtrack_count:
        print(f'{n=}; {seed=}; {rep0=}; {method=}')
        print(f'{max_backtracked_edges=}')
        print(f'{max_backtracked_edges_all=}')
        print(f'{max_backtrack_count=}')
    prev_max_backtrack_count = max_backtrack_count

print('DONE')

0 / 1000
4 / 1000
8 / 1000
12 / 1000
16 / 1000
20 / 1000
24 / 1000
28 / 1000
32 / 1000
36 / 1000
40 / 1000
44 / 1000
48 / 1000
n=9; seed=657537469; rep0=50; method='knuth'
max_backtracked_edges=[(4, 6), (5, 4)]
max_backtracked_edges_all=[((4, 6), (5, 4))]
max_backtrack_count=1
52 / 1000
56 / 1000
60 / 1000
64 / 1000
68 / 1000
72 / 1000
76 / 1000
80 / 1000
84 / 1000
88 / 1000
92 / 1000
96 / 1000
100 / 1000
n=9; seed=819324251; rep0=101; method='knuth'
max_backtracked_edges=[(2, 4), (8, 4), (2, 4), (8, 4), (2, 4), (8, 4), (2, 4), (8, 4), (2, 4), (8, 4)]
max_backtracked_edges_all=[((2, 4), (8, 4)), ((8, 4), (2, 4)), ((2, 4), (8, 4)), ((8, 4), (2, 4)), ((2, 4), (8, 4)), ((8, 4), (2, 4)), ((2, 4), (8, 4)), ((8, 4), (2, 4)), ((2, 4), (8, 4))]
max_backtrack_count=9
104 / 1000
108 / 1000
112 / 1000
116 / 1000
120 / 1000
124 / 1000
128 / 1000
132 / 1000
136 / 1000
140 / 1000
144 / 1000
148 / 1000
152 / 1000
156 / 1000
160 / 1000
164 / 1000
168 / 1000
172 / 1000
176 / 1000
180 / 1000
184 / 1000


KeyboardInterrupt: 

In [ ]:
print(f'{n=}; {seed=}; {rep0=}; {method=}')

In [ ]:
# needs "check_connectivity=False"
# n=9; seed=602740735; rep0=367; method='knuth'; rep1=1
# edge_idx=15
# worst 0 9 (4, 7)
# path=[2, 0, 5, 6, 3, 8]
# path=[7, 0, 2, 3, 6, 5, 4]
# path=[4, 3, 1, 5, 2, 7, 6]
# path=[0, 4, 2, 7, 8, 3, 5]
# path=[7, 3, 5, 1, 8]
# path=[6, 8, 7, 1, 3]
# path=[1, 8, 6, 7, 5, 4, 3]
# path=[2, 4, 0, 7, 5]
# path=[1, 7, 3, 2, 5, 0]


# breaks v1,v2 trick!
# n=11; seed=848962982; rep0=91; rep1=1
# edge_idx=12
# worst 0 0 (8, 10)
# path=[0, 2, 6, 10, 7]
# path=[6, 8, 5]
# path=[4, 8, 9, 3, 5]
# path=[4, 7, 3, 5, 8, 2]
# path=[9, 8, 4, 7, 2]
# path=[7, 8, 6, 3, 2, 1]
# path=[8, 1, 2, 6, 3, 7, 10, 0]
# path=[1, 9, 3]
# path=[9, 1, 8, 7, 2, 10]
# path=[6, 10, 2, 3]
# path=[8, 2, 0, 10]



n=6; seed=453421617; rep0=127; method='knuth'; rep1=1
edge_idx=1
worst 1 4 (4, 3)
path=[0, 5, 3]
path=[3, 2, 4]
path=[4, 2]
path=[1, 3, 2, 5]
path=[1, 3, 5, 2]
path=[0, 5]


n=6; seed=128415401; rep0=193; method='knuth'; rep1=1
edge_idx=3
worst 1 3 (3, 5)
path=[5, 0, 3, 2]
path=[3, 2, 0]
path=[4, 5, 2, 0, 3]
path=[1, 5, 4, 2]
path=[1, 5, 2, 4]
path=[5, 0]


n=11; seed=191981348; rep0=624; method='knuth'; rep1=1
edge_idx=5
worst 1 4 (2, 4)
path=[4, 0, 2, 10]
path=[9, 2, 8]
path=[9, 2, 8, 6]
path=[7, 8, 6]
path=[7, 8, 5]
path=[2, 6, 5, 8]
path=[5, 6, 2, 10, 0]
path=[3, 4, 10, 0, 2]
path=[1, 4, 3, 10]
path=[1, 4, 10, 3]
path=[4, 0]


n=10; seed=430720555; rep0=438; method='knuth'; rep1=1
edge_idx=7
worst 1 5 (5, 6)
path=[6, 2, 0, 5, 7]
path=[9, 7, 1, 2, 0]
path=[2, 6, 7, 0, 5, 9]
path=[4, 6, 1, 3]
path=[4, 6, 3, 1]
path=[3, 6, 1, 5]
path=[2, 1, 5, 9, 7, 6]
path=[8, 5]
path=[8, 5, 7]
path=[0, 7, 1]


n=10; seed=423945431; rep0=178; method='knuth'; rep1=1
edge_idx=6
worst 1 4 (7, 5)
path=[0, 6, 8]
path=[3, 5, 8, 1]
path=[3, 5, 1, 8]
path=[4, 5, 1, 7]
path=[9, 4, 2]
path=[9, 4, 1, 7, 6, 5]
path=[5, 8, 2, 7]
path=[2, 7, 6]
path=[6, 8, 2, 4, 1]
path=[0, 6, 5, 4]


n=10; seed=234681895; rep0=416; method='knuth'; rep1=1
edge_idx=6
worst 1 4 (5, 0)
path=[0, 6, 1, 5]
path=[3, 1, 9, 7, 6]
path=[7, 3, 5, 2]
path=[8, 5, 1]
path=[8, 5, 3]
path=[9, 7, 3, 1, 0]
path=[9, 1, 6, 7]
path=[4, 0, 6, 2]
path=[4, 0, 2, 6]
path=[1, 0, 2, 5]


n=11; seed=572040645; rep0=120; method='knuth'; rep1=1
edge_idx=8
worst 1 3 (2, 9)
path=[0, 9, 3, 10]
path=[4, 8, 3, 6, 5]
path=[2, 7, 5, 4]
path=[2, 1, 10, 9, 3]
path=[1, 10, 4, 5]
path=[1, 2, 7]
path=[10, 4, 7, 5, 8]
path=[6, 5, 8, 3, 7]
path=[6, 3, 7, 4, 9]
path=[8, 4, 9]
path=[0, 9, 10, 3]





# n=11; seed=878450016; rep0=121; method = 'knuth'; rep1=1
# edge_idx=12
# worst 0 1 (1, 6)
# path=[10, 2, 8, 0]
# path=[7, 1, 10, 3, 5, 8]
# path=[5, 8, 7]
# path=[5, 3, 7, 8, 6, 4]
# path=[8, 1, 7, 3, 4, 6]
# path=[2, 10, 3, 1]
# path=[10, 1, 4, 0, 2, 8, 6, 3]
# path=[0, 6, 3, 1, 9]
# path=[9, 6, 0, 8, 1]
# path=[3, 4, 1, 9, 6]
# path=[4, 0, 2]




# breaks backtracking only in part2 (or, no-backtracks-in-part1)!
# n=11; seed=784757882; rep0=35; rep1=1
# edge_idx=22
# worst 0 1 (2, 3)
# path=[3, 5, 7, 4, 0, 2, 9, 10]
# path=[2, 8, 0, 3, 9, 5, 10, 7]
# path=[2, 9, 4, 3, 5]
# path=[10, 8, 2, 0]
# path=[9, 10, 6, 5, 1, 4, 0, 8]
# path=[8, 5, 4, 1, 7, 10, 3, 0]
# path=[1, 5, 4, 3, 6]
# path=[9, 4, 10, 5, 6, 3, 7]
# path=[6, 10, 0, 5, 9, 3, 1, 7, 4]
# path=[5, 7, 3, 1, 0, 10, 4]
# path=[1, 0, 5, 8, 10, 3]


# n=9; seed=991535567; rep0=10; rep1=1
# edge_idx=6
# worst 0 1 (2, 4)
# path=[7, 5, 4, 3, 8, 6, 0, 2]
# path=[0, 6, 2, 8]
# path=[4, 5, 2, 1]
# path=[5, 7, 3]
# path=[7, 3, 8]
# path=[3, 4, 6]
# path=[4, 6, 1, 2]
# path=[1, 6, 8, 2, 5]
# path=[6, 2, 0]

# n=11; seed=168723644; rep0=75; rep1=1
# edge_idx=18
# worst 0 1 (9, 10)
# path=[4, 9, 7, 1, 5, 3, 2, 0, 8]
# path=[4, 6, 7, 10, 5, 9]
# path=[5, 7, 6, 3, 9, 1]
# path=[7, 1, 4, 6]
# path=[2, 9, 4, 1, 10, 8, 3]
# path=[6, 8, 10, 2, 5, 9, 0]
# path=[1, 10, 0]
# path=[3, 9, 2, 1, 5, 7, 10]
# path=[2, 5, 3, 6, 8]
# path=[10, 2, 3, 8, 0, 9, 7]
# path=[5, 10, 0, 2, 1, 9]

# n=11; seed=39432623; rep0=54; rep1=1
# edge_idx=29
# worst 0 1 (7, 8)
# path=[7, 0, 6, 5, 10, 9, 1, 8]
# path=[2, 8, 4, 7, 5, 10, 6, 9, 0, 1, 3]
# path=[10, 0, 5, 8, 2, 7, 6]
# path=[4, 7, 10, 2, 6, 3]
# path=[7, 9, 2, 10, 8, 3, 4, 6, 0]
# path=[6, 9, 8, 3, 7, 0, 5, 4]
# path=[2, 9, 3, 5, 6, 4, 8, 0, 1]
# path=[10, 8, 5, 7, 1, 3, 9]
# path=[9, 7, 6, 3, 4, 0]
# path=[8, 9, 1, 7, 10, 0, 4, 5]
# path=[1, 8, 0, 9, 10, 6, 2, 7, 3, 5]

# n=11; seed=768573361; rep0=587; rep1=1
# edge_idx=9
# worst 0 1 (4, 10)
# path=[6, 2, 5, 10, 0]
# path=[8, 0, 6, 2, 4, 9]
# path=[2, 5]
# path=[7, 9, 10, 5, 4]
# path=[4, 8, 1, 3, 9, 7, 10]
# path=[1, 8, 0]
# path=[8, 4, 9, 1, 3, 10]
# path=[3, 9, 1]
# path=[9, 10, 3]
# path=[7, 10, 0, 4, 5]
# path=[6, 0, 4, 2]


# n=11; seed=902180484; rep0=73; rep1=1


In [424]:
# n = 5
# n = 6
# n = 7
n = 8
# n = 9
# n = 10
# n = 11
# n = 12
# n = 13
# n = 14
# n = 15
# n = 20
# n = 30

verbose = False
# verbose = True

print_stats = True
# print_stats = False

backtrack_iteration = 20

# max_rep = 10000
max_rep = 1000
# max_rep = 100
# max_rep = 10
# max_rep = 1

rep1 = 1

prev_max_backtrack_count = 0
max_backtrack_count = 0
max_backtracked_edges = []
max_backtracked_edges_all = []

for rep0 in range(max_rep):
#     if (rep0 % 100 == 0):
    if (rep0 % 10 == 0):
#     if (rep0 % 1 == 0):
        print(rep0, '/', max_rep)
#     n = random.randint(5, 25)
    seed = random.randint(0, 1000000000)

    # hard cases examples:
#     n=8; seed=987200965; rep0=75  # in some sense, needs v1, v2 = v2, v1
    # FIXME: needs backtrack

#     n=10; seed=537501769; rep0=2395  # iteration_count = 5354; also, doesn't backtrack

    # "checked_switches" breaks the edge bruteforce (in part1? or part2?)
    # i've increased the count in "checked_switches" to 2, it helped
#     n=7; seed=167814967; rep0=71

    # maybe needs several backtracks; probably not
    # probably not: maybe we need to choose carefully to which edge to backtrack
#     n=8; seed=19975338; rep0=938

#     n=8; seed=367481793; rep0=2; rep1+=1
#     n=8; seed=987200965; rep0=75; rep1+=1

#     n=7; seed=561894248; rep0=46  # v1,v2

#     print(f'{n=}; {seed=}; {rep0=}')
    random.seed(seed)

    # 0. generate random order of edges
    edges = gen_edges(n)
    # 1. so that we actually start with building a tree
    tree_edges, non_tree_edges = sort_edges_to_build_a_tree(edges, n, verbose=verbose)
#     non_tree_edges.sort()
    random.seed(seed + rep0)
    for edge_idx, (orig_v1, orig_v2) in enumerate(non_tree_edges):
        if random.random() > 0.5:
            non_tree_edges[edge_idx] = (orig_v2, orig_v1)

    tmp_random_generator = random.Random(seed + rep0 + rep1)

    # 2. build EPPDC for this tree
    tree = Graph(n=n, edges=tree_edges)
    tree_eppdc = build_tree_eppdc(tree, n)
    assert tree_eppdc.is_eppdc()
    if verbose:
        for path in tree_eppdc:
            print(path)

    # now we iteratively add an edge
    # steps 4.-6.
    eppdc = deepcopy(tree_eppdc)

    for edge_idx, (orig_v1, orig_v2) in enumerate(non_tree_edges):
        # FIXME (debug)
#         if edge_idx == 20:
#             verbose = True

        eppdc_before_part1 = deepcopy(eppdc)
        v1, v2 = orig_v1, orig_v2

        # FIXME: remove + configure properly
        iteration_count = 0
        backtracked_edges = [(v1, v2)]
        backtracked_edges_all = []
        backtracked_once = False
        use_backtrack1 = False
        use_backtrack2 = False
        common_vertex = None

        while True:  # FIXME: we need an explicit mechanism to not have an infinite loop
            success1 = State.FAIL
            success2 = State.FAIL

            iteration_count += 1
            if (iteration_count > backtrack_iteration):
#             if not backtracked_once and (iteration_count > backtrack_iteration):
                use_backtrack1 = False
                use_backtrack2 = True
                # FIXME: we can use, but don't need a backtrack in part1
#                 if iteration_count % 2 == 0:
#                     use_backtrack1 = True
#                     use_backtrack2 = False
#             if iteration_count > 10 * backtrack_iteration:
#                 break

#             if not backtracked_once:
#             v1, v2 = v2, v1  # looks like we need this; even when we backtracked

            eppdc = deepcopy(eppdc_before_part1)

            if verbose:
                print(f'{edge_idx=} before part 1; adding edge {(v1, v2)}')
                for path in eppdc:
                    print(path)

            part1_solutions = defaultdict(set)
            for (tv1, tv2) in [(v1, v2), (v2, v1)]:
                part1_count_iteration_count = 0
                while (part1_count_iteration_count < 100):
                    part1_count_iteration_count += 1
                    eppdc = deepcopy(eppdc_before_part1)
                    tmp_success1, _ = extend_with_backtrack(n, tv1, tv2, eppdc, 1,
                                                            use_backtrack=False,
                                                            verbose=False,
                                                            rnd_gen=tmp_random_generator)
                    if tmp_success1 == State.SUCCESS:
                        part1_solutions[(tv1, tv2)].add(eppdc)
                        if len(part1_solutions[(tv1, tv2)]) > 5:
                            break
            if len(part1_solutions[(v1, v2)]) < 1 or len(part1_solutions[(v2, v1)]) < 1:
                print(f'{n=}; {seed=}; {rep0=}; {rep1=}')
                print(f'{edge_idx=}')
                print('lens', len(part1_solutions[(v1, v2)]), len(part1_solutions[(v2, v1)]))

#             success1, _ = extend_with_backtrack(n, v1, v2, eppdc, 1,
#                                                 verbose=verbose)

#             part1_iteration_count = 0
            while success1 == State.FAIL:
#                 part1_iteration_count += 1
#                 if part1_iteration_count > 2000:
#                     assert False
                eppdc = deepcopy(eppdc_before_part1)
                success1, (new_v1_p1, new_v2_p1) = extend_with_backtrack(n, v1, v2, eppdc, 1,
                                                                         use_backtrack=use_backtrack1,
                                                                         verbose=verbose)

                if verbose:
                    print(f'{edge_idx=} part1 success: {success1}')
                    if success1 != State.FAIL:
                        print(f'{success1=}')
                        print(f'{edge_idx=} after part 1;')
                        for path in eppdc:
                            print(path)
                        print(f'adding edge {(v2, v1)}')

                if success1 == State.FAIL:
                    v1, v2 = v2, v1  # looks like we need this; even when we backtracked

            assert (success1 != State.FAIL)
#                 success2, _ = extend_with_backtrack(n, v2, v1, eppdc, 2,
#                                                     verbose=verbose)

            eppdc_after_part1 = deepcopy(eppdc)  # TODO: reuse it

#             part2_iteration_count = 0
#             while success2 == State.FAIL:
#                 part2_iteration_count += 1
#                 if part2_iteration_count > 2000:
#                     assert False

            eppdc = deepcopy(eppdc_after_part1)
            success2, (new_v2_p2, new_v1_p2) = extend_with_backtrack(n, v2, v1, eppdc, 2,
                                                                     use_backtrack=use_backtrack2,
                                                                     verbose=verbose)
#             else:
#                 success2 = State.FAIL

            if success2 == State.FAIL:
                success1 = State.FAIL

            if verbose:
                print(f'{edge_idx=} part2 success: {success2}')    
                if success2 == State.SUCCESS:
                    print(f'{edge_idx=} after part 2')
                    for path in eppdc:
                        print(path)
                time.sleep(0.1)

            if success2 != State.FAIL:
                assert eppdc.is_connected()
                assert eppdc.is_eppdc()

                if success1 == State.SUCCESS and success2 == State.SUCCESS:
                    # assert not use_backtrack1 and not use_backtrack2  # FIXME: ideally this should be the case
                    backtrack_count = len(list(backtracked_edges)) - 1
                    if backtrack_count >= max_backtrack_count:
                        max_backtrack_count = backtrack_count
                        max_backtracked_edges = backtracked_edges
                        max_backtracked_edges_all = backtracked_edges_all
#                         print(f'{edge_idx=}')
#                         print(f'{max_backtracked_edges=}')
#                         print(f'{max_backtracked_edges_all=}')
#                         print(f'{max_backtrack_count=}')
                    break

                assert (success1 == State.BACKTRACK or success2 == State.BACKTRACK)
                if success1 == State.BACKTRACK:
                    new_v1, new_v2 = new_v1_p1, new_v2_p1
                else:
                    new_v1, new_v2 = new_v1_p2, new_v2_p2
                new_edge = (new_v1, new_v2)
                if common_vertex is None:
                    common_vertex = set(backtracked_edges[0]) & set(new_edge)
                    assert len(common_vertex) == 1
                    common_vertex = list(common_vertex)[0]

                if new_v1 != common_vertex and new_v2 != common_vertex:
                    continue

#                 if (new_v1, new_v2) in backtracked_edges and backtracked_edges[-2] == (new_v1, new_v2):
#                     backtracked_edges = backtracked_edges[:-1]
#                 else:
                backtracked_edges.append((new_v1, new_v2))
                backtracked_edges_all.append(((v1, v2), (new_v1, new_v2)))

                if verbose:
                    print(f'{edge_idx=} backtrack edge ({v1}, {v2}); now try ({new_v1}, {new_v2})')
                    print('')

                v1, v2 = new_v1, new_v2
                eppdc_before_part1 = deepcopy(eppdc)
                iteration_count = 0
                use_backtrack1 = False
                use_backtrack2 = False
                backtracked_once = True
            else:
                if verbose:
                    print('fail, try again')
                    print('')

        if success2 == State.FAIL:
            print(f'{n=}; {seed=}; {rep0=}')
            print(f'{success1=}, {success2=}, {edge_idx=} vs {len(non_tree_edges)}')
            print(f'(v1, v2) = ({v1}, {v2})')
            for path in eppdc_before_part1:
                print(path)
            assert False
        assert eppdc.is_eppdc()

    if max_backtrack_count > prev_max_backtrack_count:
        print(f'{n=}; {seed=}; {rep0=}')
        print(f'{max_backtracked_edges=}')
        print(f'{max_backtracked_edges_all=}')
        print(f'{max_backtrack_count=}')
    prev_max_backtrack_count = max_backtrack_count

print('DONE')

0 / 1000
10 / 1000
20 / 1000
30 / 1000
n=8; seed=8244698; rep0=37; rep1=1
edge_idx=10
lens 3 0
n=8; seed=8244698; rep0=37; rep1=1
edge_idx=10
lens 3 0
n=8; seed=8244698; rep0=37; rep1=1
edge_idx=10
lens 3 0
n=8; seed=8244698; rep0=37; rep1=1
edge_idx=10
lens 3 0
n=8; seed=8244698; rep0=37; rep1=1
edge_idx=10
lens 3 0
40 / 1000
n=8; seed=145046696; rep0=45; rep1=1
edge_idx=20
lens 0 6
n=8; seed=145046696; rep0=45; rep1=1
edge_idx=20
lens 6 0
n=8; seed=145046696; rep0=45; rep1=1
edge_idx=20
lens 6 0
n=8; seed=145046696; rep0=45; rep1=1
edge_idx=20
lens 6 0
n=8; seed=145046696; rep0=45; rep1=1
edge_idx=20
lens 6 0
n=8; seed=145046696; rep0=45; rep1=1
edge_idx=20
lens 6 0
n=8; seed=145046696; rep0=45; rep1=1
edge_idx=20
lens 6 0
n=8; seed=145046696; rep0=45; rep1=1
edge_idx=20
lens 6 0
n=8; seed=145046696; rep0=45; rep1=1
edge_idx=20
lens 6 0
50 / 1000
60 / 1000
70 / 1000
80 / 1000
90 / 1000
100 / 1000
110 / 1000
120 / 1000
130 / 1000
140 / 1000
150 / 1000
160 / 1000
170 / 1000
180 / 1000


n=8; seed=370223562; rep0=891; rep1=1
edge_idx=16
lens 3 0
n=8; seed=370223562; rep0=891; rep1=1
edge_idx=16
lens 5 0
n=8; seed=370223562; rep0=891; rep1=1
edge_idx=16
lens 5 0
n=8; seed=370223562; rep0=891; rep1=1
edge_idx=16
lens 5 0
n=8; seed=370223562; rep0=891; rep1=1
edge_idx=16
lens 4 0
n=8; seed=370223562; rep0=891; rep1=1
edge_idx=16
lens 3 0
n=8; seed=370223562; rep0=891; rep1=1
edge_idx=16
lens 5 0
n=8; seed=370223562; rep0=891; rep1=1
edge_idx=16
lens 4 0
n=8; seed=370223562; rep0=891; rep1=1
edge_idx=16
lens 5 0
n=8; seed=370223562; rep0=891; rep1=1
edge_idx=16
lens 4 0
n=8; seed=370223562; rep0=891; rep1=1
edge_idx=16
lens 5 0
n=8; seed=370223562; rep0=891; rep1=1
edge_idx=16
lens 4 0
n=8; seed=370223562; rep0=891; rep1=1
edge_idx=16
lens 4 0
n=8; seed=370223562; rep0=891; rep1=1
edge_idx=16
lens 5 0
n=8; seed=370223562; rep0=891; rep1=1
edge_idx=16
lens 5 0
n=8; seed=370223562; rep0=891; rep1=1
edge_idx=16
lens 5 0
n=8; seed=370223562; rep0=891; rep1=1
edge_idx=16
lens 4

In [ ]:
# lens 2 0
# n=7; seed=18363301; rep0=920; rep1=1
# n=7; seed=736492902; rep0=18; rep1=1
# n=7; seed=70889700; rep0=680; rep1=1

# lens 3 0
# n=7; seed=970694693; rep0=294; rep1=1
# n=7; seed=771723231; rep0=317; rep1=1
# n=7; seed=676363995; rep0=955; rep1=1

# lens 4 0
# n=7; seed=304158294; rep0=777; rep1=1

# n=7; seed=561894248; rep0=46
# n=7; seed=996991329; rep0=60
# n=7; seed=808882777; rep0=72; rep1=1
# n=7; seed=249320750; rep0=156; rep1=1
# n=7; seed=144894745; rep0=265; rep1=1
# n=7; seed=668090230; rep0=266; rep1=1
# n=7; seed=223846543; rep0=419; rep1=1
# n=7; seed=473324142; rep0=536; rep1=1
# n=7; seed=817231038; rep0=873; rep1=1


In [54]:
print(f'{n=}; {seed=}; {rep0=}')
# TODO
# n=8; seed=656158194; rep0=63
# n=10; seed=350991691; rep0=9



# max_backtracked_edges_all=[((4, 1), (2, 4))]



# n=6; seed=151869693; rep0=8902
# max_backtracked_edges=[(5, 3), (0, 5), (4, 5)]

# n=6; seed=458587679; rep0=4466
# max_backtracked_edges=[(2, 1), (4, 1), (0, 1)]

# n=7; seed=646643421; rep0=370
# max_backtracked_edges=[(0, 6), (4, 6), (2, 6)]

# n=8; seed=273581813; rep0=8665
# max_backtracked_edges=[(5, 3), (1, 5), (3, 5), (6, 5), (1, 5)]

# DONE
# n=20; seed=729444865; rep0=798
# max_backtracked_edges=[(6, 15), (7, 6), (1, 7), (4, 7), (6, 7)]
# =>
# max_backtracked_edges=[(6, 15), (7, 6)]

# n=20; seed=247503912; rep0=281
# max_backtracked_edges=[(10, 14), (8, 14), (6, 14), (15, 14), (8, 14)]


n=11; seed=784757882; rep0=35
